In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
A = np.array([0.4, 1.2, 0.5, 1.7, 0.2, 1.6, 0.2, 1.4, 1.9]) # interarrival times
S = np.array([2.0, 0.7, 0.2, 1.1, 3.7, 0.6]) # service times
# these would typically be generated during the simulation run

# initialization routine
simulation_clock = 0
server_status = 0 # idle
number_in_queue = 0
times_of_arrival = []
time_of_last_event = 0

# event list gives the times of the next occurrence of each of the event types
event_list = {
    'A': 0 + A[0], # first arrival
    'D': np.inf, # first event obviously cannot be a departure
}

# four statistical counters
states = {
    'num_delayed': 0,
    'total_delay': 0,
    'area_under_q_t': 0,
    'area_under_b_t': 0,
}

In [ ]:
# arrival of customer 1
# pass control to the arrival routine

if server_status == 0: 
    # if idle, begin service immediately (no delay)
    D = 0
    server_status = 1 # now busy, but queue still empty

# implementation

In [ ]:
import numpy as np

def expon(mean): 
    return np.random.exponential(scale=mean)

In [147]:
NEXT_EVENT_TYPE = 0
NUM_CUSTS_DELAYED = 0
NUM_DELAYS_REQUIRED = 1_000
NUM_EVENTS = 2
NUM_IN_Q = 0
SERVER_STATUS = 0

AREA_NUM_IN_Q = 0
AREA_SERVER_STATUS = 0
MEAN_INTERARRIVAL = 0
MEAN_SERVICE = 0
SIM_TIME = 0
TIME_ARRIVAL = []
TIME_LAST_EVENT = 0
TIME_NEXT_EVENT = [0,0]
TOTAL_OF_DELAYS = 0

MEAN_INTERARRIVAL = 1.0
MEAN_SERVICE = 0.50

from enum import auto, Enum

class ServerStatus(Enum):
    IDLE = 0
    BUSY = 1


def initialize():
    """Initialize simulation variables"""
    # init sim clock
    global SIM_TIME
    SIM_TIME = 0.0
    
    # init state vars
    global SERVER_STATUS, NUM_IN_Q, TIME_LAST_EVENT
    SERVER_STATUS = ServerStatus.IDLE
    NUM_IN_Q = 0
    TIME_LAST_EVENT = 0.0

    # init stat counters
    global NUM_CUSTS_DELAYED, TOTAL_OF_DELAYS, AREA_NUM_IN_Q, AREA_SERVER_STATUS
    NUM_CUSTS_DELAYED = 0
    TOTAL_OF_DELAYS = 0.0
    AREA_NUM_IN_Q = 0.0
    AREA_SERVER_STATUS = 0.0

    # init event list. No customers are present, so departure event is impossible
    global TIME_NEXT_EVENT, MEAN_INTERARRIVAL
    TIME_NEXT_EVENT[0] = SIM_TIME + expon(MEAN_INTERARRIVAL)
    TIME_NEXT_EVENT[1] = np.inf
    


def timing():
    """Compare the timings of the next of each possible event type to find
    the next event. Set NEXT_EVENT_TYPE accordingly, and advance the sim
    clock to the time of occurrent of this event.
    """
    global NEXT_EVENT_TYPE, NUM_EVENTS, TIME_NEXT_EVENT, SIM_TIME

    # min_time_next_event = np.inf

    # for i in range(NUM_EVENTS):
    #     if TIME_NEXT_EVENT[i] < min_time_next_event:
    #         min_time_next_event = TIME_NEXT_EVENT[i]
    #         NEXT_EVENT_TYPE = i

    if TIME_NEXT_EVENT[0] < TIME_NEXT_EVENT[1]:
        # arrival is next
        NEXT_EVENT_TYPE = 0
        SIM_TIME = TIME_NEXT_EVENT[0]
    else:
        NEXT_EVENT_TYPE = 1
        SIM_TIME = TIME_NEXT_EVENT[1]
    


def arrive():
    """Handle the arrival of a new customer.
    If server is busy, add customer to queue. If server is idle, customer is
    immediately serviced with a delay of 0.
    """
    # print('arriving')
    delay = 0

    # schedule the next arrival
    global TIME_NEXT_EVENT, SIM_TIME, MEAN_INTERARRIVAL
    TIME_NEXT_EVENT[0] = SIM_TIME + expon(MEAN_INTERARRIVAL)

    global SERVER_STATUS
    if SERVER_STATUS is ServerStatus.BUSY:
        # if busy, go to the queue
        global NUM_IN_Q
        NUM_IN_Q += 1

        # check for queue overflow
        if NUM_IN_Q > 100:
            raise Exception("Queue has overflowed!")
        
        # store arrival time of arriving customer at the end of the queue
        global TIME_ARRIVAL
        TIME_ARRIVAL += [SIM_TIME]
    else:
        # if idle, go directly to service
        delay = 0.0
        global TOTAL_OF_DELAYS, NUM_CUSTS_DELAYED
        TOTAL_OF_DELAYS += delay

        # increment number of customers delayed, make server busy
        NUM_CUSTS_DELAYED += 1
        SERVER_STATUS = ServerStatus.BUSY

        # schedule a departure
        global MEAN_SERVICE
        TIME_NEXT_EVENT[1] = SIM_TIME + expon(MEAN_SERVICE)

def depart():
    """Handle the departure of a customer from service."""
    # print('departing')

    global NUM_IN_Q
    if NUM_IN_Q == 0:
        # queue is empty, so server becomes idle and departures are impossible
        global SERVER_STATUS, TIME_NEXT_EVENT
        SERVER_STATUS = ServerStatus.IDLE
        TIME_NEXT_EVENT[1] = np.inf
    else:
        # there are customers in the queue
        NUM_IN_Q -= 1

        # compute delay of the customer who is starting service
        global SIM_TIME, TIME_ARRIVAL, TOTAL_OF_DELAYS
        delay = SIM_TIME - TIME_ARRIVAL[0]
        TOTAL_OF_DELAYS += delay

        # increment number of customers delayed, schedule next departure
        global NUM_CUSTS_DELAYED, MEAN_SERVICE
        NUM_CUSTS_DELAYED += 1
        TIME_NEXT_EVENT[1] = SIM_TIME + expon(MEAN_SERVICE)

        # move customers up in queue
        TIME_ARRIVAL = TIME_ARRIVAL[1:]

def report():
    """Log statistics"""
    global TOTAL_OF_DELAYS, NUM_CUSTS_DELAYED, AREA_NUM_IN_Q, SIM_TIME, AREA_SERVER_STATUS
    print(f"\n\nAverage delay in queue {TOTAL_OF_DELAYS / NUM_CUSTS_DELAYED:11.3f} minutes\n\n")
    print(f"Average number in queue {AREA_NUM_IN_Q / SIM_TIME:10.3f}\n\n")
    print(f"Server utilization {AREA_SERVER_STATUS / SIM_TIME:15.3f}\n\n")
    print(f"Time simulation ended {SIM_TIME:12.3f} minutes")
    

def update_time_avg_stats():
    """Prior to processing each event, this function updates the areas tracked
    for the calculation of continuous-time statistics."""
    global SIM_TIME, TIME_LAST_EVENT
    time_since_last_event = SIM_TIME - TIME_LAST_EVENT
    TIME_LAST_EVENT = SIM_TIME # current time

    # update area under num-in-queue function
    global AREA_NUM_IN_Q, NUM_IN_Q
    AREA_NUM_IN_Q += NUM_IN_Q * time_since_last_event

    # update area under server-busy indicator function
    global AREA_SERVER_STATUS, SERVER_STATUS
    AREA_SERVER_STATUS += SERVER_STATUS.value * time_since_last_event

def main():
    # initialize
    initialize()

    # run the simulation until all delays are observed
    while (NUM_CUSTS_DELAYED < NUM_DELAYS_REQUIRED):
        # determine the next event
        timing()
        # global NEXT_EVENT_TYPE
        # print(f"next event is of type {NEXT_EVENT_TYPE}")
        # update time-average statistical accumulators
        update_time_avg_stats()

        # invoke the appropriate event function
        if NEXT_EVENT_TYPE == 0:
            arrive()
        else:
            depart()

    # invoke the report generator and end the simulation
    report()

    return
    


In [148]:
main()



Average delay in queue       0.429 minutes


Average number in queue      0.406


Server utilization           0.493


Time simulation ended     1055.788 minutes
